# CYCLING IN SYDNEY

In [5]:
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
import math
from datetime import datetime

In [6]:
%matplotlib inline

In [7]:
plt.rcParams['figure.figsize'] = (10, 5)

In [8]:
saved_style_state = matplotlib.rcParams.copy()

## 1. importing data

In [9]:
if os.path.isfile("Cycleway Counts_01Jan18_11Sept20.xlsx"):
    filepath = "Cycleway Counts_01Jan18_11Sept20.xlsx"
    print("loading from file")
else:
    filepath = "https://opendata.transport.nsw.gov.au/system/files/resources/Cycleway%20Counts_01Jan18_11Sept20.xlsx"
    print("loading from the internet")

cycleway_usage = pd.read_excel(filepath, sheet_name="daily counts")
print("done")


loading from file
done


# 1.a ben's assistance

In [19]:
dir_lookup = ["North", "East", "South", "West", "All Directions"]
def get_mode(index):
    if index<5:
        return "Cycling"
    else:
        return "Pedestrian"

locations_sparse = [x for x in cycleway_usage.columns if "Unnamed" not in x ]

In [20]:
def get_loc(i, response_type="all"):
    try:
        loc = locations_sparse[math.ceil(i/15)]
        if response_type == "all":
            return loc
        elif response_type == "code":
            return loc.split("-")[0].strip()
        elif response_type == "name":
            return loc.split("-")[1].strip()
        # You can extend this to pull out the notes, like (Cycleway and Pedestrian counter), but they're pretty inconsistent
        else:
            print(f"`{response_type}` is not implemented yet")
    except:
        print("er")


In [21]:
new_rows = []
for i, row in cycleway_usage.iterrows():
    if i < 3:
        continue  # This feels nasty

    date = row[0]
    for loc_index in range(1, len(row) - 15, 15):
        section = row[
            loc_index : loc_index + 10
        ]  # 10 because we don't care about the Sum (All Transport Modes) section
        for col_index, col in enumerate(section):
            dir = dir_lookup[col_index % 5]
            if type(col) is int and col != 0 and dir != "All Directions":
                r = {
                    "date": date,
                    "location_name": get_loc(loc_index, "name"),
                    "station": get_loc(loc_index, "code"),
                    "mode": get_mode(col_index),
                    "direction": dir,
                    "count": int(col),
                }
                new_rows.append(r)

# 1.b converting datetime

In [22]:
time_df = pd.DataFrame(new_rows)
time_df['date'] = pd.to_datetime(time_df['date'], infer_datetime_format=True)
time_df.sort_values(by='date', inplace=True) 

C:\Users\Bojana.Pavlovic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '13/01/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Bojana.Pavlovic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '14/01/2018' in DD/MM/YYYY format. Provide format or specify infer_datetime_format=True for consistent parsing.
  cache_array = _maybe_cache(arg, format, cache, convert_listlike)
C:\Users\Bojana.Pavlovic\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pandas\core\tools\datetimes.py:1047: UserWarning: Parsing '15/01/2018' in DD/MM

In [23]:
print(time_df.shape)
time_df.head() 

(43329, 6)


,date,location_name,station,mode,direction,count
1467,2018-01-02,"Captain Cook Bridge cycleway, near Taren Point...",90913,Cycling,North,64
1457,2018-01-02,"Liverpool to Parramatta Rail Trail, Railway Te...",90905,Cycling,North,58
1455,2018-01-02,"Liverpool to Parramatta Rail Trail, Broomfield...",90904,Cycling,North,20
1454,2018-01-02,"Anzac Bridge cycleway, near Quarry Master Driv...",90903,Pedestrian,West,159
1453,2018-01-02,"Anzac Bridge cycleway, near Quarry Master Driv...",90903,Pedestrian,East,159


### Questions
* Who is travelling into the city, out of the city?
* Weekdays vs weekend? See comparison, poor connections? Limitations of only pathways/single counter?

## 2. data visualisation over time
* per day/ per month bar chart
* section off weekends?
* weekdays?

## 3. data visualisation linked to location
* plot points on a map
* set size relation to counter (int)
* set colour/arrow relation to counter/direction
* where is there a lot of cycling? 

In [24]:
if os.path.isfile("cyclecounters-Oct-2019.csv"):
    counterfilepath = "cyclecounters-Oct-2019.csv"
    print("loading from file")
else:
    counterfilepath = "https://opendata.transport.nsw.gov.au/node/6771/download"
    print("loading from the internet")

cycleway_counterlocation = pd.read_csv(counterfilepath)
print("done")

loading from file
done


In [29]:
cycleway_counterlocation.sort_values(by='station', inplace=True)
cycleway_counterlocation.head(26)

,objectid,station,region,geolocation,location,shortname,notes,y_2018_oct,y_2018_nov,y_2018_dec,...,y_2012,y_2013,y_2014,y_2015,y_2016,y_2017,y_2018,y_2019,y_max_year,active
8,9,30011,Metro Sydney Inner West,-33.860980 151.164268,"Iron Cove Bridge cycleway, near King George Pa...",Iron Cove Bridge,Interruptions due to construction work 2010/11...,0,0,0,...,552,563,646,603,627,563,504,400,800,Y
0,1,90902,Sydney City,-33.858959 151.206218,"Sydney Harbour Bridge bicycle path, near Upper...",Sydney Harbour Bridge,None,1744,1974,1549,...,1807,1932,1853,1671,1729,1707,1663,1464,2000,Y
1,2,90903,Sydney City,-33.867926 151.181554,"Anzac Bridge cycleway, near Quarry Master Driv...",Anzac Bridge,Counter inoperable from Feb 2015 to Oct 2015.,1249,1433,1122,...,1414,1457,1376,1111,1236,1223,1208,1090,1500,Y
21,22,90904,Metro Sydney West Central,-33.89126 150.94075,"Liverpool to Parramatta Rail Trail, Broomfield...",Liverpool to Paramatta Rail Trail Cabramatta,Closed in 2012 due to work on Southern Sydney ...,25,0,0,...,23,0,0,31,33,33,28,32,50,Y
22,23,90905,Metro Sydney West Central,-33.851949 150.986339,"Liverpool to Parramatta Rail Trail, Railway Te...",Liverpool to Parramatta Rail Trail Guildford,Counter inoperable Mar 2014 to June 2015.,69,73,70,...,79,80,86,64,69,61,71,63,120,Y
2,3,90907,Sydney City,-33.89582 151.221849,"Anzac Parade cycleway, near Lang Road, Moore Park",Anzac Parade,Counter inoperable December 2013 to June 2015.,812,895,927,...,943,1029,1145,983,979,929,831,571,1100,N
7,8,90908,Metro Sydney Inner North,-33.817990 151.089939,"North Ryde to Sydney Olympic Park cycleway, ne...",North Ryde to Sydney Olympic Park,Counter inoperable June 2015 to Nov 2015.,379,376,377,...,358,417,423,452,284,551,431,433,500,Y
25,26,90910,Metro Sydney South,-33.9874 151.07716,"Como Bridge cycleway, near Oatley Parade, Oatley",Como Bridge,2010/11 - cycleway partially inaccessible due ...,129,116,129,...,291,287,253,241,236,232,213,144,850,NaN
12,13,90911,Metro Sydney North,-33.76469 151.105899,"Lane Cove River Walk, near Browns Waterhole, M...",Lane Cove River Walk,None,0,0,0,...,140,168,199,176,180,170,178,0,180,N
13,14,90912,Metro Sydney North West,-33.75367 150.966419,"Crestwood Reserve cycleway, near Merindah Road...",Crestwood Reserve,None,0,1,1,...,48,50,61,59,32,36,49,6,90,N


## 4. next steps?
* import datasets from Strava, Google Maps, layer over data
* understand why cycling is used: recreation/commuting?
* understand why cycling is *NOT* used
* start playing with cycle pathways?
* feed into research: making Parramatta road a cycleway